In [ ]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.io as sio
from functools import reduce 
import scipy.io as sio
from scipy.signal import medfilt
import h5py
from py_ecog_utils.helpers import *
import numpy as np
import sys
from segment_EEG import *
import re 
import mne

from fooof import FOOOF

import neurodsp
# Import spectral power functions
from neurodsp.spectral import compute_spectrum, rotate_powerlaw

# Import utilities for loading and plotting data
from neurodsp.utils import create_times
from neurodsp.utils.download import load_ndsp_data
from neurodsp.plts.spectral import plot_power_spectra
from neurodsp.plts.time_series import plot_time_series
from neurodsp.filt import filter_signal

In [ ]:
datadir = "/wynton/protected/home/amorim/djcald/processed_data/filtered_data"
bn = 'MZSsRYfhk398gZnZRZdu'
filename = f"{datadir}/preprocessed_raw_Patient_{bn}.h5"
filename

chans = [x for x in file_data['raw'].keys() if 'ECoG' in x and 'ECoGA' not in x]
raw_data = np.array([file_data['raw'][k] for k in chans])
fs = 256
times = create_times(raw_data.shape[1]/fs, fs)
plot_duration = 4 # in hours
ch_names = chans
times.shape

In [ ]:
#iterate through chunks of the recording
for start_idx in np.arange(0, raw_data.shape[1], plot_duration*3600*fs):
    end_idx = min(raw_data.shape[1], start_idx + plot_duration*3600*fs)
    #Grab relevant section of data
    raw_seg = raw_data[:,start_idx:end_idx]

# subset of good data 
raw_seg = raw_data[:,500000:3800000]
raw_seg = raw_seg[:,100000:150000]
raw_seg.shape


In [ ]:
plot_time_series(create_times(raw_seg.shape[1]/fs,fs),raw_seg)

In [ ]:
filter_range = (58,62)
notch_data = filter_signal(raw_seg,fs,'bandstop',filter_range,n_seconds=0.5)
freq_mean, psd_mean = compute_spectrum(raw_seg, fs, method='welch', avg_type='mean', nperseg=fs*2)


In [ ]:
nrow = 7
ncol = 1
fig, axs = plt.subplots(nrow, ncol)
for i, ax in enumerate(fig.axes):
    ax.loglog(freq_mean,psd_mean[i,:])

#plt.savefig('powerspectra.png')

In [ ]:
nrow = 7
ncol = 1
fig, axs = plt.subplots(nrow, ncol)
for i, ax in enumerate(fig.axes):
    ax.plot(create_times(raw_seg.shape[1]/fs,fs),raw_seg[i,:])

#plt.savefig('timeseries.png')

plot_power_spectra(freq_mean,psd_mean)
#plt.savefig('powerspectralltogether.png')

In [ ]:
plt.figure()

ac_seg = raw_seg
ac_scale = 5000
space_between = 0.5

n_x_ticks = 5
ac_clr_opt = "k" #AC band color (black)

ac_axis = plt.subplot(111)
ac_upper = ac_scale*2*len(ch_names)+(len(ch_names)+1)*(ac_scale*2*space_between)
ac_axis.set_ylim([0, ac_upper])
ac_axis.set_yticks([])
ac_axis.set_xticks([])
ac_axis.spines['top'].set_visible(False)
ac_axis.spines['right'].set_visible(False)
ac_axis.spines['bottom'].set_visible(False)

#Configure x-axis tickmarks
xtick_reallocs = np.arange(0, plot_duration*3600*fs+1, plot_duration*3600*fs/(n_x_ticks-1)).astype(int)
xticklocs = np.arange(start_idx, end_idx+1, (end_idx-start_idx)/(n_x_ticks-1)).astype(int)
ac_axis.set_xticks(xtick_reallocs)
ac_axis.set_xticklabels([round(x, 2) for x in xticklocs/(fs*3600)])
ac_axis.set_xlabel('Time since file start (hr)')

#plot scalebars on topmost channel
ac_axis.axvline(x=plot_duration*3600*fs*1.01, ymin=(ac_upper-ac_scale*2)/ac_upper, ymax=(ac_upper-ac_scale)/ac_upper, linewidth=2, color=ac_clr_opt)
ac_axis.annotate(str(round(ac_scale)) + " μV", (plot_duration*3600*fs*1.015, (ac_upper-ac_scale*1.75)), color=ac_clr_opt)

#Center channels along y-axis
ac_y_centers = np.arange(ac_scale*2, ac_upper,ac_scale*2+ac_scale*2*space_between)
ac_y_centers_dict = dict(zip(np.flip(ch_names), ac_y_centers))
ac_axis.set_yticks(ac_y_centers)
ac_axis.set_yticklabels(np.flip(ch_names))

#plot black ac signal in background
for i, ch in enumerate(ch_names):
    ac_axis.plot(ac_seg[i,:]+ac_y_centers_dict[ch], color=ac_clr_opt, linewidth=0.5)


In [ ]:
fooof_total = []

# Define model settings
peak_width = [1, 8]             # `peak_width_limit` setting
n_peaks = 6                     # `max_n_peaks` setting
peak_height = 0.10              # `min_peak_height` setting

# Define frequency range
PSD_range = [30, 45]

for idx in list(range(0,psd_mean.shape[0])):
    # Initialize a model object for spectral parameterization, with some settings
    fm = FOOOF(peak_width_limits=peak_width, max_n_peaks=n_peaks, min_peak_height=peak_height, verbose=False)
    
    # Fit individual PSD over 3-40 Hz range
    fm.report(freq_mean, psd_mean[idx,:], PSD_range)

    plt.savefig(f"fooof{idx}")
    
    # Access the model fit parameters from the model object
    print('Aperiodic parameters: \n', fm.aperiodic_params_, '\n')
    print('Peak parameters: \n', fm.peak_params_, '\n')
    print('Goodness of fit:')
    print('Error - ', fm.error_)
    print('R^2   - ', fm.r_squared_, '\n')
    print('Number of fit peaks: \n', fm.n_peaks_)
    
    fooof_output = fm
    fooof_total.append(fooof_output)